#Load Data
We have data from MMLLAB DeepFashion 1 attribute prediction dataset
It has 1000 attribute classes with each image having multiple labels from different attribute classes

We have data as zip file in drie we are extracting it here
We have labels as column adjacency list which we are converting to complete binary multi label encoding and creating dataloaders

In [2]:
from zipfile import ZipFile
with ZipFile('/content/drive/MyDrive/data/wardrobe_helper_data/Category and Attribute Prediction Benchmark/Img/img.zip', 'r') as archive:
  archive.extractall('/content/wardrobe_img')

In [3]:
if __name__ == '__main__' :   
    from fastai.vision.all import *
    # from timeit import default_timer
    # import torch
    # import torchvision
    # from torchvision import transforms
    import os, multiprocessing
    import pandas as pd

# do define multiprocessing related things in this 'if' block

    multiprocessing.freeze_support()            # needed for handling multiprocessing 
    
    device = 'cuda' if torch.cuda.is_available() else 'cpu'         # setting up device
    print(device)
    
    drive = r'/content/drive/MyDrive/data/wardrobe_helper_data/'
    ###################################################
    ########### reading up labels from disk ############
    ##################################################
    
    ## reading file containing attribute names
    desc = pd.read_csv(drive + r'Anno_coarse/list_attr_cloth.txt', skiprows = 2, header = None)
    desc = desc.iloc[:,0].str.extract('(.*)[ ]{2,}([12345])')
    
    ## generating boolean dataframe of labels from compressed representation
    labels = pd.DataFrame(False,                                                                            # initializing all labels as False
                          columns = desc.iloc[:, 0].str.strip().values,                                     # setting column names as attribute names
                          index = pd.read_csv(drive + 'attribute_labels_index.csv', header = None).iloc[:, 0])      # setting indexes as img names
    
    # In 'attribute_labels_colwise.csv' each row 'i' contains list of row numbers
    # which will be 'true' in 'labels' df in column 'i'
    for column_num_labels, row in pd.read_csv(drive + 'attribute_labels_colwise.csv').iterrows():         
        labels.iloc[row.dropna().astype(int), column_num_labels] = True
    #print(labels.info())
    
    
    # split_df stores information about splitting imgs in 3 sets
    split_df = pd.read_excel(drive + "list_eval_partition.xlsx").set_index('image_name')

    test_labels = labels.loc[split_df.loc[split_df.evaluation_status == 'test'].index]
    train_val_labels = labels.loc[split_df.loc[split_df.evaluation_status != 'test'].index]

    train_val_labels.loc[:, 'is_valid'] = False
    train_val_labels.loc[split_df.loc[split_df.evaluation_status == 'val'].index, 'is_valid'] = True
    val_index = np.arange(train_val_labels.shape[0])[train_val_labels.is_valid.values]
    
    base_img_dir = Path('wardrobe_img')

    def item_getter(df) :
        #print(df.head(), '\n', df.shape)
        ret = (list(map(lambda x : base_img_dir / x, df.index)), df.iloc[:, :-1].values)
        #print(ret)
        return ret
    print('dataloaders')
    
    dblocks = DataBlock.from_columns(blocks = (ImageBlock, MultiCategoryBlock(encoded = True, vocab = list(labels.columns.values))),
                                     get_items = item_getter,
                                     splitter = IndexSplitter(val_index),
                                     item_tfms = Resize(224, pad_mode = 'zeros'),
                                     batch_tfms = Normalize.from_stats([0.485, 0.456, 0.406], [0.229, 0.224, 0.225])
                                    )

    # del desc, labels, split_df, train_val_labels, val_index
    

cuda
dataloaders


#Loss Function 
Defining custom loss function which applies weight to classes as we have only 4 labels of 1000 for each data point

So we adjust positive and negative classes using weights to prevent negative labels (0 labels) dominate the loss and model prediction

In [4]:
if __name__ == "__main__"  :
    dls = dblocks.dataloaders(train_val_labels, bs = 64, num_workers = 2)
    print(dls.device)
    wp = train_val_labels.values.mean()
    my_extra_cb = Callback(before_fit = lambda self : self.learn.model.to(device, non_blocking = True))
    loss_fn = BCEWithLogitsLossFlat(reduction = 'none')
    
    wp = (1 - wp) / wp
    def my_loss(ypred, ytrue) :
        #print(ypred, ytrue)
        loss = loss_fn(ypred, ytrue)
        loss[ytrue.ravel() == 1] *= wp
        return loss.mean() * (1 + 1 / wp) / 2 


cuda:0


#Training model using ResNet50 architecture

In [ ]:
if __name__ == '__main__' :
    learn = vision_learner(dls, resnet50, loss_func = my_loss)
    #print(learn.dls.after_batch, learn.dls.device) 
    learn.fine_tune(1, freeze_epochs = 1)

#Some metric calculations

In [ ]:
# dls.vocab[dls.valid_ds[0][1] == 1]

(#4) ['chic','pleated','sleeveless','v-neck']

In [ ]:
yp = preds[0] >= 0.5
y = preds[1] 

In [ ]:
((yp == y).to(float)).mean() 

TensorMultiCategory(0.9159, dtype=torch.float64)

In [ ]:
yp.sum() / y.sum()

TensorMultiCategory(25.9916)

## exporting model

In [ ]:
learn.export(drive + 'models/attr_3_weighted_unfreeze.pkl') 

In [ ]:
pd.DataFrame([[1,0],[0,1]]).values.mean()

0.5

In [ ]:
from sklearn.metrics import confusion_matrix

In [ ]:
confusion_matrix(y.ravel(), yp.ravel())

array([[36527649,  3339741],
       [   25602,   107008]])

# Final accuracy of model

In [7]:
from fastai.vision.all import *
drive = r'/content/drive/MyDrive/data/wardrobe_helper_data/'
# learn = vision_learner(dls, resnet50, loss_func = my_loss) 
learn = load_learner(drive + 'models/attr_3_weighted_unfreeze.pkl', cpu = False)

In [8]:
preds, targs = learn.get_preds(dl = dls.valid)[:2]
preds = preds >= 0.5

def acc(p, y) :
  return ((p == y).to(float)).mean() 

print('Accuracy of attribute predicter is: ', acc(preds, targs))

Accuracy of attribute predicter is:  TensorMultiCategory(0.9159, dtype=torch.float64)
